In [ ]:
from pathlib import Path

from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brpylib
from time import time

In [ ]:
filename = '20190122-065346-004'

datafile = f'data/{filename}.ns6'
datapath = Path(datafile)

start_time_s = 1.0
plot_chan = 1

elec_ids     = list(range(1, 97))      # 'all' is default for all (1-indexed)
data_time_s  = 'all' # 2.0 # 'all' # 2.0                     # 'all' is default for all
downsample   = 1        

In [ ]:
# Open file and extract headers
nsx_file = brpylib.NsxFile(str(datapath))

# Extract data - note: data will be returned based on *SORTED* elec_ids, see cont_data['elec_ids']
cont_data = nsx_file.getdata(elec_ids, start_time_s, data_time_s, downsample, full_timestamps=True)

# Close the nsx file now that all data is out
nsx_file.close()

## RAW

In [ ]:
# Plot the data channel
seg_id = 0
ch_idx  = cont_data["elec_ids"].index(plot_chan)
t = cont_data["data_headers"][seg_id]["Timestamp"] / cont_data["samp_per_s"]

In [ ]:
channel_data_1 = cont_data["data"][seg_id][0]
channel_data_3 = cont_data["data"][seg_id][2]
channel_data_9 = cont_data["data"][seg_id][8]
# channel_data_29 = cont_data["data"][seg_id][29]

print(np.shape(channel_data_1))
channel_data_1 = channel_data_1 * 0.01
print('channel_1')
print(channel_data_1[100])
print(channel_data_1[1000])
print('channel_2')
print(channel_data_3[100])
print(channel_data_3[1000])
print('channel_9')
print(channel_data_9[100])
print(channel_data_9[1000])

# data = {'channel_1':channel_data}
# df = pd.DataFrame(data)

# # Write to CSV
# df.to_csv('signal.csv', index=False)

In [ ]:
plt.plot(t, cont_data["data"][seg_id][ch_idx])
plt.axis([t[0], t[-1], min(cont_data["data"][seg_id][ch_idx]), max(cont_data["data"][seg_id][ch_idx])])
plt.locator_params(axis="y", nbins=20)
plt.xlabel("Time (s)")
# plt.ylabel("Output (" + nsx_file.extended_headers[hdr_idx]['Units'] + ")")
# plt.title(nsx_file.extended_headers[hdr_idx]['ElectrodeLabel'])
plt.show()

In [ ]:
print(len(cont_data["data"][seg_id][ch_idx][:100]))

# Direct Neural Biasing - PySignalProcessor

In [ ]:
import direct_neural_biasing as dnb

In [ ]:
# Create an instance of the signal processor
processor = dnb.PySignalProcessor(logging=True, downsampling_rate=1)

# Add a BandPass filter
processor.add_filter(id="butterworth", f0=100.0, fs=10000.0)

# Add threshold detectors
processor.add_threshold_detector(id="det1", filter_id="butterworth", threshold=1.0, buffer_size=100, sensitivity=0.1)
processor.add_threshold_detector(id="det2", filter_id="butterworth", threshold=2.0, buffer_size=100, sensitivity=0.2)

# Add a pulse trigger
processor.add_pulse_trigger(id="main_trigger", activation_detector_id="det1", inhibition_detector_id="det2", activation_cooldown=10, inhibition_cooldown=5)

# Process the signal
results = processor.run(cont_data["data"][seg_id][ch_idx])

# Step 4: Output detection results
# for result in results:
#     print(result)

print('\n')
print(results[0])

## Scipy

In [ ]:
b, a = signal.butter(2, [0.25, 4], 'bp', fs=cont_data["samp_per_s"])
scipy_signal = signal.lfilter(b,a,cont_data["data"][seg_id][ch_idx])

In [ ]:
plt.plot(t, scipy_signal)
plt.axis([t[0], t[-1], min(scipy_signal), max(scipy_signal)])
plt.locator_params(axis="y", nbins=20)
plt.xlabel("Time (s)")
# plt.ylabel("Output (" + nsx_file.extended_headers[hdr_idx]['Units'] + ")")
# plt.title(nsx_file.extended_headers[hdr_idx]['ElectrodeLabel'])
plt.show()